In [21]:
import folium
import pandas as pd

In [22]:
from folium.plugins import MarkerCluster
from folium.plugins import MousePosition
from folium.features import DivIcon

In [23]:
URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
spacex_df = pd.read_csv(URL)

In [24]:
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launchsite_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launchsite_df = launchsite_df[['Launch Site', 'Lat', 'Long']]
launchsite_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


In [25]:
nasa_cordinates = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_cordinates, zoom_start=10)


In [26]:
circle = folium.Circle(nasa_cordinates, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
marker = folium.map.Marker(nasa_cordinates, icon=DivIcon(icon_size=(20,20), icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
))
site_map.add_child(circle)
site_map.add_child(marker)

In [27]:
SLC_coordinates = [28.563197,	-80.576820]
KSC_coordinates = [28.573255,	-80.646895]
VAFB_coordinates = [34.632834,	-120.610745]
CC_coordinates = [28.562302,	-80.577356]



circle2 = folium.Circle(CC_coordinates, radius=20000, color='#d35400', fill=True).add_child(folium.Popup('CCAFS LC-40'))
marker2 = folium.map.Marker(CC_coordinates, icon=DivIcon(icon_size=(20, 20), icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'CCAFS LC-40'
))

site_map.add_child(circle2)
site_map.add_child(marker2)


circle3 = folium.Circle(KSC_coordinates, radius = 20000, color = '#00FF00', fill=True).add_child(folium.Popup('KSC LC-39A'))
marker3 = folium.map.Marker(KSC_coordinates, icon=DivIcon(icon_size=(20, 20), icon_anchor=(0,0), html='<div style="font-size: 12; color:#00FF00;"><b>%s</b></div>' % 'KSC LC-39A'))

site_map.add_child(circle3)
site_map.add_child(marker3)

circle4 = folium.Circle(VAFB_coordinates, radius = 20000, color = '#0000FF', fill=True).add_child(folium.Popup('VAFB SLC-4E'))
marker4 = folium.map.Marker(VAFB_coordinates, icon=DivIcon(icon_size=(20, 20), icon_anchor=(0,0), html='<div style="font-size: 12; color:#0000FF;"><b>%s</b></div>' % 'VAFB SLC-4E'))

site_map.add_child(circle4)
site_map.add_child(marker4)

In [28]:
marker_cluster = MarkerCluster()
spacex_df['marker_color'] = ['green' if i == 1 else 'red' for i in spacex_df['class']]
spacex_df.tail(10)

,Launch Site,Lat,Long,class,marker_color
46,KSC LC-39A,28.573255,-80.646895,1,green
47,KSC LC-39A,28.573255,-80.646895,1,green
48,KSC LC-39A,28.573255,-80.646895,1,green
49,CCAFS SLC-40,28.563197,-80.576820,1,green
50,CCAFS SLC-40,28.563197,-80.576820,1,green
51,CCAFS SLC-40,28.563197,-80.576820,0,red
52,CCAFS SLC-40,28.563197,-80.576820,0,red
53,CCAFS SLC-40,28.563197,-80.576820,0,red
54,CCAFS SLC-40,28.563197,-80.576820,1,green
55,CCAFS SLC-40,28.563197,-80.576820,0,red


In [29]:
site_map.add_child(marker_cluster)

for i, record in spacex_df.iterrows():
    lat = record['Lat']
    lon = record['Long']
    marker_cluster.add_child(folium.map.Marker([lat, lon], icon=folium.Icon(color=record['marker_color'])))

site_map

In [30]:
formatter = "function(num) {return L.Util.formatNum(num, 5);}"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

In [36]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c

    return distance

In [39]:
distance_coastline = calculate_distance(28.562302, -80.577356, 29.00000, -80.87311)

In [41]:
coast_marker = folium.map.Marker([29.00000, -80.87311], icon=DivIcon(icon_size=(20, 20), icon_anchor=(0, 0), html='<div style="font-size: 12; color:#000000;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_coastline)))

site_map.add_child(coast_marker)

In [43]:
line = folium.PolyLine(locations=[[28.562302, -80.577356], [29.00000, -80.87311]], weight=5)
site_map.add_child(line)

In [44]:
distance_city = calculate_distance(28.562302, -80.577356, 27.64461, -81.6394)

In [45]:
city_marker = folium.map.Marker([27.64461, -81.6394], icon=DivIcon(icon_size=(20, 20), icon_anchor=(0, 0), html='<div style="font-size: 12; color:#000000;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_city)))

site_map.add_child(city_marker)

In [46]:
city_line = folium.PolyLine(locations=[[28.562302, -80.577356], [27.64461, -81.6394]], weight=5)
site_map.add_child(city_line)